In [1]:
import pandas as pd
from app_logger.logger import app_logger
from datetime import datetime
import shutil
import os
import re
import json
pd.set_option('display.Max_columns',None)

In [2]:
class RawDataValidation:
    def __init__(self,raw_data_directory):
        self.logger = app_logger()
        self.raw_data_directory = raw_data_directory


    def create_data_directory(self):
        try:
            if not os.path.isdir("Data/"):
                os.makedirs("Data/")
            if not os.path.isdir("Train_logs/"):
                os.makedirs("Train_logs/")
        except OSError as o:
            file = open("Train_Logs/directorylog.txt","a+")
            self.logger.log(file,f'OSError: Error in directory creation {o}')
            file.close()
            raise OSError
        

    def raw_files_validations(self):
         pass
        
    def extract_column_data(self,column_file):                                             
            with open(column_file,'r') as file:
                lines = file.readlines()
                
            column_names= {}
            pattern= re.compile(r'^\s*(.*):\s*(.*)\.$')
            for line in lines:
                match=pattern.match(line)
            
                if match:
                    att = match.group(1)
                    value = [value.strip() for value in match.group(2).split(",")]
                    column_names[att] = value
                    
            return column_names
    
    def Data_file_validation(self,data_file):
    
            data= pd.read_csv(data_file,header=None)
            no_need_column = data.iloc[:,-1:]
            data = data.drop(no_need_column,inplace=True, axis=1)

            return data

    def combine_row_column(self,data_file,column_file):
            column_name = self.extract_column_data(column_file)
            data = self.Data_file_validation(data_file)
            if column_name and data is not None:
                data.columns = column_name.keys()
                data_file_path = os.path.join(self.raw_data_directory, "combined_data.csv")
                data.to_csv(data_file_path, index=False)
                shutil.copy(data_file_path, "Data/")
               
        

In [3]:
raw_data_dir = r'C:\Users\49179\Desktop\Machine Learning Projects\Thyroid Detection\raw_data'
obj = RawDataValidation(raw_data_dir)
obj.create_data_directory()
column_ = os.path.join(raw_data_dir, "thyroid0387.names")
record = os.path.join(raw_data_dir, "thyroid0387.data")
obj.combine_row_column(record,column_)


In [4]:
data = pd.read_csv('Data\Final_train_data.csv')

In [5]:
data.head()

,age,sex,on_thyroxine,query_on_thyroxine,on_antithyroid_medication,sick,pregnant,thyroid_surgery,I131_treatment,query_hypothyroid,query_hyperthyroid,lithium,goitre,tumor,hypopituitary,psych,TSH_measured,TSH,T3_measured,T3,TT4_measured,TT4,T4U_measured,T4U,FTI_measured,FTI,TBG_measured,TBG,referral_source,Class
0,29,F,f,f,f,f,f,f,f,t,f,f,f,f,f,f,t,0.3,f,?,f,?,f,?,f,?,f,?,other,-[840801013]
1,29,F,f,f,f,f,f,f,f,f,f,f,f,f,f,f,t,1.6,t,1.9,t,128,f,?,f,?,f,?,other,-[840801014]
2,41,F,f,f,f,f,f,f,f,f,t,f,f,f,f,f,f,?,f,?,f,?,f,?,f,?,t,11,other,-[840801042]
3,36,F,f,f,f,f,f,f,f,f,f,f,f,f,f,f,f,?,f,?,f,?,f,?,f,?,t,26,other,-[840803046]
4,32,F,f,f,f,f,f,f,f,f,f,f,f,f,f,f,f,?,f,?,f,?,f,?,f,?,t,36,other,S[840803047]


In [6]:
data.shape

(9172, 30)

In [7]:
data.columns

Index(['age', 'sex', 'on_thyroxine', 'query_on_thyroxine',
       'on_antithyroid_medication', 'sick', 'pregnant', 'thyroid_surgery',
       'I131_treatment', 'query_hypothyroid', 'query_hyperthyroid', 'lithium',
       'goitre', 'tumor', 'hypopituitary', 'psych', 'TSH_measured', 'TSH',
       'T3_measured', 'T3', 'TT4_measured', 'TT4', 'T4U_measured', 'T4U',
       'FTI_measured', 'FTI', 'TBG_measured', 'TBG', 'referral_source',
       'Class'],
      dtype='object')

In [8]:
import numpy as np
num = len(data['Class'].unique())

for i in range(num):
    pattern = r'[A-Za-z]+'
    if str(data['Class'][i]).startswith('-'):
        data['Class'][i]= "Nagetive"
    else:
        x = re.findall(pattern,data['Class'][i])
        data['Class'][i] = ''.join(x)
        
df = data.copy()

C:\Users\49179\AppData\Local\Temp\ipykernel_24692\2067485902.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['Class'][i]= "Nagetive"
C:\Users\49179\AppData\Local\Temp\ipykernel_24692\2067485902.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['Class'][i] = ''.join(x)


In [9]:
df['Class'].unique()

array(['Nagetive', 'S', 'F', 'AK', 'R', 'I', 'M', 'N', 'G', 'K', 'A',
       'KJ', 'L', 'MK', 'Q', 'J', 'CI', 'O', 'LJ', 'HK', 'D', 'GK', 'MI',
       'P', 'FK', 'B', 'GI', 'C', 'GKJ', 'OI', 'DR', 'E'], dtype=object)

In [10]:
class RawFilesValidation:
    def __init__(self,raw_data_directory):
        self.raw_data_directory = raw_data_directory
        self.logger = app_logger()
        self.schema = 'schema_training.json'

    def train_dataset(self,data_file):
        with open(self.schema,'r') as f:
            name = json.load(f)
            f.close()
        column_names = name['ColName']
        number_of_columns = name['NumberofColumns']
        data = pd.read_csv(data_file,header=None)
        if data and column_names:
            data.columns = self.column_names.keys()
            data_file_path = os.path.join(self.raw_data_directory,"final_train_data.csv")
            data.to_csv(data_file_path,index=False)
            shutil.move(data_file_path,'Data/')
        

In [11]:
file = 'schema_training.json'
with open(file,'r') as f:
            name = json.load(f)
            f.close()
column_names = name['ColName']
column_names.keys()

dict_keys(['age', 'sex', 'on_thyroxine', 'query_on_thyroxine', 'on_antithyroid_medication', 'sick', 'pregnant', 'thyroid_surgery', 'I131_treatment', 'query_hypothyroid', 'query_hyperthyroid', 'lithium', 'goitre', 'tumor', 'hypopituitary', 'psych', 'TSH_measured', 'TSH', 'T3_measured', 'T3', 'TT4_measured', 'TT4', 'T4U_measured', 'T4U', 'FTI_measured', 'FTI', 'TBG_measured', 'TBG', 'referral_source', 'Class'])

In [12]:
import sqlite3
connection = sqlite3.connect("./training.db")
cursor = connection.cursor()
cursor.execute("SELECT count(name) from sqlite_master WHERE type = 'table' and name = 'GoodRawData'")
if cursor.fetchone()[0] == 1:
    connection.close()
else:
    for keys in column_names.keys():
        types = column_names[keys]
        try:
            cursor.execute("ALTER TABLE GoodRawData ADD COLUMN '{Cname}' {dtypes}".format(Cname = keys, dtypes = types))
        except:
            cursor.execute("CREATE TABLE GoodRawData({Cname} {dtypes})".format(Cname = keys, dtypes = types))

In [ ]:
import csv
connection = sqlite3.connect("./training.db")
cursor = connection.cursor()
fi = [files for files in os.listdir(r'Data/')]
keys = [key for key in column_names.keys()]
path = 'Data/'
for file in fi:
    with open (path+file,'r') as f:
        next(f)
        read = csv.reader(f,delimiter='\n')
        for l in read:
            row_values = l[0].split(",")
            cursor.execute('INSERT INTO GoodRawData VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)',
                           row_values)
            connection.commit()
connection.close()
                



In [28]:
connection = sqlite3.connect("./training.db")
cursor = connection.cursor()
sql_query = 'SELECT * FROM GoodRawData'
cursor.execute(sql_query)
result = cursor.fetchall()
header = [i[0] for i in cursor.description]
csv_file = csv.writer(open('extra.csv','w',newline=""),delimiter=',', lineterminator='\r\n',quoting=csv.QUOTE_ALL, escapechar='\\')
csv_file.writerow(header)
csv_file.writerows(result)


In [27]:
cursor.description

(('age', None, None, None, None, None, None),
 ('sex', None, None, None, None, None, None),
 ('on_thyroxine', None, None, None, None, None, None),
 ('query_on_thyroxine', None, None, None, None, None, None),
 ('on_antithyroid_medication', None, None, None, None, None, None),
 ('sick', None, None, None, None, None, None),
 ('pregnant', None, None, None, None, None, None),
 ('thyroid_surgery', None, None, None, None, None, None),
 ('I131_treatment', None, None, None, None, None, None),
 ('query_hypothyroid', None, None, None, None, None, None),
 ('query_hyperthyroid', None, None, None, None, None, None),
 ('lithium', None, None, None, None, None, None),
 ('goitre', None, None, None, None, None, None),
 ('tumor', None, None, None, None, None, None),
 ('hypopituitary', None, None, None, None, None, None),
 ('psych', None, None, None, None, None, None),
 ('TSH_measured', None, None, None, None, None, None),
 ('TSH', None, None, None, None, None, None),
 ('T3_measured', None, None, None, Non

In [1]:
x= 'abc'
x.replace('a','b')

'bbc'

In [8]:
import pandas as pd
csv = pd.read_csv(r'C:\Users\49179\Desktop\Machine Learning Projects\Thyroid Detection\Data\Final_train_data.csv')
columns = ['sex', 'on_thyroxine', 'query_on_thyroxine', 'on_antithyroid_medication', 'sick', 'pregnant',
                           'thyroid_surgery', 'I131_treatment', 'query_hypothyroid', 'query_hyperthyroid', 'lithium',
                           'goitre', 'tumor', 'hypopituitary', 'psych', 'TSH_measured', 'T3_measured', 'TT4_measured',
                           'T4U_measured', 'FTI_measured', 'TBG_measured', 'TBG', 'referral_source', 'Class']
for column in csv.columns:
    if column in columns:
        csv[column] = csv[column].apply(lambda x: "'" + str(x) + "'")

In [9]:
csv.columns

Index(['age', 'sex', 'on_thyroxine', 'query_on_thyroxine',
       'on_antithyroid_medication', 'sick', 'pregnant', 'thyroid_surgery',
       'I131_treatment', 'query_hypothyroid', 'query_hyperthyroid', 'lithium',
       'goitre', 'tumor', 'hypopituitary', 'psych', 'TSH_measured', 'TSH',
       'T3_measured', 'T3', 'TT4_measured', 'TT4', 'T4U_measured', 'T4U',
       'FTI_measured', 'FTI', 'TBG_measured', 'TBG', 'referral_source',
       'Class'],
      dtype='object')